### Modelo

Notebook destinado al entrenamiento de la U-Net con el conjunto de datos inicial

In [40]:
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [41]:
DIR_TRAIN_DATA =  os.path.join(os.getcwd(), "..", "data", "preprocessed_train")
NUM_CLASSES  = 10

In [42]:
def npz_generator():
    """
    Generador usado por el pipeline de tf para la obtención de imágenes y sus máscaras desde el directorio donde se aloja el cto de datos
    """
    for file_name in os.listdir(DIR_TRAIN_DATA):
        if file_name.endswith(".npz"):
            file_path = os.path.join(DIR_TRAIN_DATA, file_name)
            with np.load(file_path) as data:
                x = data['image']  # Replace 'instance' with your actual key for input
                y = data['mask']         # Replace 'y' with your actual key for labels
                yield x, y

In [43]:
output_signature = (
    tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32),  # `x`: 3-channel images (e.g., RGB)
    tf.TensorSpec(shape=(None, None, NUM_CLASSES), dtype=tf.float32)    # `y`: 2D segmentation mask (integer class labels)
)


In [44]:
dataset = tf.data.Dataset.from_generator(
    npz_generator,
    output_signature=output_signature
)

In [45]:
def preprocess(x, y):
    # Resize images and labels to the same size
    x = tf.image.resize(x, (256, 256))  # Resize input images
    y = tf.image.resize(y, (256, 256))  # Resize target masks
    return x, y

In [46]:
for x, y in dataset.take(1):  # Take the first element of the dataset
    print("Input shape (x):", x.shape)
    print("Target shape (y):", y.shape)

Input shape (x): (480, 640, 3)
Target shape (y): (480, 640, 10)


In [47]:
batch_size = 32
dataset = dataset.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [48]:
# Define a U-Net model
def unet_model(input_shape=(None, None, 3), num_classes=5):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Encoder
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u1 = tf.keras.layers.UpSampling2D((2, 2))(c3)
    u1 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    u1 = tf.keras.layers.Concatenate()([u1, c2])

    u2 = tf.keras.layers.UpSampling2D((2, 2))(u1)
    u2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    u2 = tf.keras.layers.Concatenate()([u2, c1])

    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(u2)

    model = tf.keras.Model(inputs, outputs)
    return model

# Create the U-Net model
model = unet_model(input_shape=(None, None, 3), num_classes=NUM_CLASSES)

In [49]:
# Define Dice Loss
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator / (denominator + tf.keras.backend.epsilon())

In [50]:
def iou_metric(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true + y_pred) - intersection
    return intersection / (union + tf.keras.backend.epsilon())

In [51]:
model.compile(
    optimizer='adam',
    loss=dice_loss,
    metrics=[iou_metric, 'accuracy']
)

In [85]:
np.max(x[0][:,:,1])

255.0

In [52]:
model.fit(dataset, epochs=20)


Epoch 1/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 5372s 47s/step - accuracy: 0.8383 - iou_metric: 1.6193 - loss: -0.0888
Epoch 2/20


c:\Users\ruben\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


112/112 ━━━━━━━━━━━━━━━━━━━━ 4946s 44s/step - accuracy: 0.9117 - iou_metric: 1.7206 - loss: -0.1705
Epoch 3/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 4876s 44s/step - accuracy: 0.9117 - iou_metric: 1.7206 - loss: -0.1705
Epoch 4/20
 88/112 ━━━━━━━━━━━━━━━━━━━━ 17:34 44s/step - accuracy: 0.9093 - iou_metric: 1.6789 - loss: -0.1604

KeyboardInterrupt: 

In [54]:
for x, y in dataset.take(1):  # Take the first element of the dataset
    print("Input shape (x):", x.shape)
    print("Target shape (y):", y.shape)


    predictions = model.predict(x)

print(predictions.shape)


Input shape (x): (32, 256, 256, 3)
Target shape (y): (32, 256, 256, 10)
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
(32, 256, 256, 10)


In [65]:
(x[0].shape)

TensorShape([256, 256, 3])

In [63]:
import matplotlib.pyplot as plt


In [64]:
color_dict = {
    0: (255, 0, 0),    # Red
    1: (0, 255, 0),    # Green
    2: (0, 0, 255),    # Blue
    3: (255, 255, 0),  # Yellow
    4: (255, 0, 255),  # Magenta
    5: (0, 255, 255),  # Cyan
    6: (128, 128, 128),# Gray
    7: (128, 0, 0),    # Maroon
    8: (0, 128, 0),    # Dark Green
    9: (0, 0, 128),    # Navy
}

In [ ]:

def get_mask_colored(mask):

    H, W = mask.shape[0], mask.shape[1]
    labeled_mask = np.argmax(mask, axis=-1)
    colored_mask = np.zeros((H, W, 3), dtype=np.uint8)
    for label, color in color_dict.items():
        colored_mask[labeled_mask == label] = color

    return colored_mask



def plot_results(image, prediction, ground_truth):

    H, W, C = image.shape[0], image.shape[1], 10

    fig , axs = plt.subplots(1, 3, figsize=(15, 5))
    axs = axs.flatten()  # Flatten the array for easier indexing
    
    axs[0].imshow(image)
    axs[0].axis('off')
    axs[0].set_title('Original')


    mask_pred = get_mask_colored(prediction)
    axs[1].imshow(mask_pred)
    axs[1].axis('off')
    axs[1].set_title('Mask predicted')


    mask_ground_truth = get_mask_colored(ground_truth)

    axs[2].imshow(mask_ground_truth)
    axs[2].axis('off')
    axs[2].set_title('Ground truth')
    


    plt.tight_layout()
    plt.show()


In [9]:
# Dice coefficient and loss
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1, 2, 3))
    union = tf.reduce_sum(y_true, axis=(1, 2, 3)) + tf.reduce_sum(y_pred, axis=(1, 2, 3))
    return tf.reduce_mean((2. * intersection + smooth) / (union + smooth), axis=0)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def log_cosh_dice_loss(y_true, y_pred):
    dice_loss_value = dice_loss(y_true, y_pred)
    return tf.reduce_mean(tf.math.log(tf.cosh(dice_loss_value)))

In [10]:

# U-Net Model (same as before)
def unet_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = models.Model(inputs, outputs)
    return model

In [11]:
# Load data
data_folder = os.path.join(os.getcwd(), "..", "data", "preprocessed_train")
input_images, masks = load_all_data(data_folder)

# Preprocess data
input_images = preprocess_data(input_images)


C:\Users\ruben\AppData\Local\Temp\ipykernel_55292\2104337063.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(input_images), np.array(masks)


ValueError: setting an array element with a sequence.

In [ ]:

print(input_images.shape)
height, width, num_channels = input_images.shape


In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_images, masks, test_size=0.2, random_state=42)

# Build and compile the model
model = unet_model(input_shape=(height, width, num_channels), num_classes=10)
model.compile(optimizer='adam', loss=log_cosh_dice_loss, metrics=[dice_coefficient, 'accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=8, epochs=50)

In [12]:
### TESTINGS 
import os
import numpy as np
import tensorflow as tf

# Define a generator function to load data from multiple .npz files
def data_generator(npz_files, batch_size):
    X_batch = []
    y_batch = []

    for npz_file in npz_files:
        with np.load(npz_file) as data:
            X = data['X']  # Input data
            y = data['y']  # Label

        # Add the current instance to the batch
        X_batch.append(X)
        y_batch.append(y)

        # Yield the batch when it reaches the desired size
        if len(X_batch) >= batch_size:
            yield np.array(X_batch), np.array(y_batch)
            X_batch, y_batch = [], []  # Reset for the next batch

    # Yield remaining instances if they do not form a complete batch
    if len(X_batch) > 0:
        yield np.array(X_batch), np.array(y_batch)


In [13]:
# Path to the directory containing your .npz files
npz_directory = "path/to/your/npz_files"  # Change this to the path of your .npz files

# Get all the .npz files in the directory
npz_files = [os.path.join(npz_directory, fname) for fname in os.listdir(npz_directory) if fname.endswith('.npz')]

batch_size = 32  # Set the batch size

# Create a TensorFlow Dataset from the generator
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(npz_files, batch_size),  # Pass the generator
    output_signature=(
        tf.TensorSpec(shape=(None, X.shape[1]), dtype=tf.float32),  # Shape of X (features)
        tf.TensorSpec(shape=(None,), dtype=tf.int32)  # Shape of y (labels)
    )
)

# Shuffle, batch, and prefetch the data
dataset = dataset.shuffle(buffer_size=1000)  # Adjust buffer size for shuffling
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'path/to/your/npz_files'

In [14]:
# Example model definition
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X.shape[1],)),  # Adjust input shape to your data
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


NameError: name 'X' is not defined

In [ ]:
model.fit(dataset, epochs=10)


### AUX



In [55]:
from tensorflow.keras import layers, Model


In [56]:
def unet_model(input_shape=(128, 128, 3), num_classes=5):
    inputs = tf.keras.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    # Output layer
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs, outputs)
    return model

In [57]:
model = unet_model(input_shape=(128, 128, 3), num_classes=5)
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_36 (Conv2D)  │ (None, 128, 128,  │      1,792 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_37 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_36[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 64, 64,    │          0 │ conv2d_37[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_8[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_39 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_38[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 32, 32,    │          0 │ conv2d_39[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 32, 32,    │    295,168 │ max_pooling2d_9[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_40[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_10    │ (None, 16, 16,    │          0 │ conv2d_41[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_10… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 16, 16,    │  2,359,808 │ conv2d_42[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 8, 8, 512) │          0 │ conv2d_43[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 8, 8,      │  4,719,616 │ max_pooling2d_11… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 8, 8,      │  9,438,208 │ conv2d_44[0][0]   │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 16, 16,    │  2,097,664 │ conv2d_45[0][0]   │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 16, 16,    │          0 │ conv2d_transpose

 Total params: 31,032,005 (118.38 MB)

 Trainable params: 31,032,005 (118.38 MB)

 Non-trainable params: 0 (0.00 B)